Partie 1 : chargement des donnees 
1.1 Reddit

In [15]:
from Author import Author
from document import document

def add_document(id2doc, id2aut, id_counter, titre, auteur, date, url, texte):
    """
    Ajoute un document au corpus et met à jour les auteurs.
    
    Retourne : nouveau id_counter (incrémenté)
    """

    # 1) Création du document
    doc = document(titre, auteur, date, url, texte)

    # 2) Ajout dans id2doc
    doc_id = f"doc_{id_counter}"
    id2doc[doc_id] = doc
    id_counter += 1

    # 3) Ajout auteur(s)
    # Si plusieurs auteurs : séparer par virgules
    author_list = [a.strip() for a in auteur.split(",")]

    for name in author_list:
        if name not in id2aut:
            id2aut[name] = Author(name)

        id2aut[name].add(doc_id, doc)

    return id_counter


In [ ]:
import praw
from datetime import datetime

reddit = praw.Reddit(
    client_id='_qZmFflED2oGl3LmqZvqBg',
    client_secret='1wQi5AamyCOBlq5HQqbpu85PDRjySQ',
    user_agent='projet_api'
)

mot_cles = [
    'climate', 'climatechange', 'environment', 'sustainability',
    'carbonfootprint', 'ecology', 'greenenergy', 'climateaction'
]

id2doc = {}
id2aut = {}
id_counter = 0

for mot in mot_cles:
    print(f"\n Searching in r/{mot} ")
    posts = reddit.subreddit(mot).top(limit=20)

    for post in posts:
        text = post.selftext.replace("\n", " ").strip()
        if text == "":
            continue

        title = post.title
        author_name = str(post.author) if post.author else "Unknown"
        date = datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d")
        url = f"https://reddit.com{post.permalink}"

        #  Ajouter le document
        id_counter = add_document(
            id2doc, id2aut, id_counter,
            titre=title,
            auteur=author_name,
            date=date,
            url=url,
            texte=text
        )

print("\nNombre de documents Reddit :", id_counter)
print("Nombre d'auteurs :", len(id2aut))



 Searching in r/climate 

 Searching in r/climatechange 

 Searching in r/environment 

 Searching in r/sustainability 

 Searching in r/carbonfootprint 

 Searching in r/ecology 

 Searching in r/greenenergy 

 Searching in r/climateaction 

Nombre de documents Reddit : 26
Nombre d'auteurs : 26


1.2 Arxiv

In [ ]:
import urllib.request
import xmltodict
from document import document
from Author import Author
from datetime import datetime
import certifi
import ssl

ssl_context = ssl.create_default_context(cafile=certifi.where())
theme = "climatechange" 
print("\n--- Requesting Arxiv ---")

url = f"https://export.arxiv.org/api/query?search_query=all:{theme}&start=0&max_results=20"
response = urllib.request.urlopen(url, context=ssl_context)
xml_data = response.read()

# Parsing XML en dictionnaire Python
data = xmltodict.parse(xml_data)
entries = data["feed"]["entry"]

# Si un seul document retourné → mettre dans une liste
if isinstance(entries, dict):
    entries = [entries]

for entry in entries:
    title = entry.get("title", "No title").strip()
    abstract = entry.get("summary", "").replace("\n", " ").strip()
    url = entry.get("id", "")
    date = entry.get("published", "unknown")

    # Récupération des noms d'auteurs
    authors = entry.get("author", [])
    if isinstance(authors, dict):
        authors = [authors]

    author_names_list = [a["name"] for a in authors]
    author_names_str = ", ".join(author_names_list)

    #  Ajouter le document
    id_counter = add_document(
        id2doc,
        id2aut,
        id_counter,
        titre=title,
        auteur=author_names_str,     # string : "Alice, Bob"
        date=date,
        url=url,
        texte=abstract
    )

print("\nNombre de documents Arxiv ajoutés :", len(entries))
print("Taille totale du corpus :", len(id2doc))
print("Nombre total d'auteurs :", len(id2aut))


Partie 2 : sauvegarde des données
2.1

In [ ]:
import os
import pandas as pd

filename = f"{theme.replace(' ', '_')}_corpus.csv"

if os.path.exists(filename):
    #  Charger directement le CSV
    df = pd.read_csv(filename, sep="\t")
    print(f" Corpus chargé depuis {filename} ({len(df)} documents)")
else:
    #  Construire le DataFrame à partir de id2doc
    data = []

    for doc_id, doc in id2doc.items():
        data.append({
            "id": doc_id,
            "titre": doc.titre,
            "auteur": doc.auteur,
            "date": doc.date,
            "url": doc.url,
            "texte": doc.texte
        })

    df = pd.DataFrame(data)

    #  Sauvegarde avec tabulation (\t)
    df.to_csv(filename, sep="\t", index=False)
    print(f" Corpus sauvegardé dans {filename}")
    print(f" Nombre total de documents : {len(df)}")


 Corpus sauvegardé dans climatechange_corpus_test.csv
 Nombre total de documents : 26


Partie 3

In [11]:
#Taille du corpus
df.shape
#Afficher le Nombre de mots et de phrase pour chaque doc
df["word_count"] = df["texte"].apply(lambda x: len(str(x).split()))
df['sentence_count']=df['texte'].apply(lambda x : len(str(x).split('.')))
df
#Supprimer les documents trop petits (moins de 20c)
df_new=df[df['word_count']>=20]
df.shape
df_new.shape


(25, 8)

Statistiques d'un auteur

In [ ]:
name = input("\nEntrez le nom d'un auteur pour obtenir ses statistiques : ")

if name in id2aut:
    aut = id2aut[name]

    # 1) Nombre de documents
    print(f"\nAuteur : {aut.name}")
    print(f"Nombre de documents produits : {aut.ndoc}")

    # 2) Taille moyenne des documents
    #    On compte le nombre de mots dans chaque texte
    total_words = sum(len(doc.texte.split()) for doc in aut.production.values())
    moyenne = total_words / aut.ndoc if aut.ndoc > 0 else 0

    print(f"Taille moyenne des documents : {moyenne:.2f} mots")

else:
    print("Auteur inconnu dans le corpus.")



Auteur : Mountain-Blacksmith7
Nombre de documents produits : 1
Taille moyenne des documents : 202.00 mots
